In [4]:
# Step 1: Install required library
# pip install scikit-surprise

from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
import pandas as pd

# Step 2: Sample interaction data (like product ratings on a shopping site)
data_dict = {
    'user_id': [
        'U1', 'U1', 'U1', 'U2', 'U2', 'U2', 'U3', 'U3', 'U4', 'U4',
        'U5', 'U5', 'U6', 'U6', 'U7', 'U7', 'U8', 'U8', 'U9', 'U9'
    ],
    'product_id': [
        'P1', 'P2', 'P5', 'P2', 'P3', 'P4', 'P1', 'P4', 'P2', 'P5',
        'P3', 'P5', 'P1', 'P6', 'P2', 'P6', 'P4', 'P5', 'P3', 'P6'
    ],
    'rating': [
        5, 4, 3, 5, 3, 4, 4, 2, 5, 3,
        4, 5, 3, 4, 2, 4, 5, 4, 3, 5
    ]
}

df = pd.DataFrame(data_dict)

# Step 3: Load into Surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['user_id', 'product_id', 'rating']], reader)

# Step 4: Train-test split
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# Step 5: Model (SVD - matrix factorization)
model = SVD()
model.fit(trainset)

# Step 6: Evaluate on testset
predictions = model.test(testset)

# Optional: Calculate RMSE (evaluation)
from surprise import accuracy
accuracy.rmse(predictions)

# Step 7: Make recommendations for a user
user_id = 'U1'
all_products = df['product_id'].unique()
rated_products = df[df['user_id'] == user_id]['product_id']
unrated_products = [p for p in all_products if p not in rated_products]

# Predict ratings for unrated products
user_predictions = [model.predict(user_id, p) for p in unrated_products]

# Sort by predicted rating
top_recommendations = sorted(user_predictions, key=lambda x: x.est, reverse=True)

print(f"\nTop recommendations for {user_id}:")
for rec in top_recommendations:
    print(f"Product: {rec.iid}, Predicted Rating: {rec.est:.2f}")

RMSE: 1.5243

Top recommendations for U1:
Product: P1, Predicted Rating: 4.29
Product: P2, Predicted Rating: 4.21
Product: P6, Predicted Rating: 4.09
Product: P3, Predicted Rating: 4.08
Product: P4, Predicted Rating: 3.98
Product: P5, Predicted Rating: 3.70
